In [1]:
import import_ipynb
import numpy as np
import math

# Astrophysics constants

In [2]:
#Unités du système international
h = 6.63E-34
c = 3.00E8
k_b = 1.38E-23
pc = 3.08E16

M_so = 1.988E30 #Masse Solaire (kg)
R_so = 6.96E8 # Rayon solaire
R_Terre = 6378.137E3 # mètres
AU = 1.50E11
RJ=7.15E7
MJ= 1.90E27
M_Terre = 5.97e24 # kg
T_Terre = 254.3 #Kelvin
TJ = 110.0 #Kelvin
rho_J = 1326 # kg/m³ Masse volumique globale de Jupiter
rho_Terre = 5515 # kg/m³ Masse volumique globale de la Terre

m2r = 4.85E-9
A_B = 0.3 # BOND ALBEDO FOR JUPITER AND EARTH

# Input parameters

Wavelength of observation (meters)

In [3]:
L = 500E-9 

Maximum limit of contrast

In [4]:
C_lim = 1E-10

Base Line of the instrument

In [5]:
B_max = 200 

Inner Working Angle (mas)

In [6]:
R = L/(2*B_max)*(1/m2r)

Outer working angle (mas)

In [7]:
OWA = 250 

Other parameters

In [8]:
S= 1e-42 # Theoritical Sensibility
T_max = 1  # Integration maximum days
Trans = 0.5 # Transmission factor of optics

# Recomputation of the Planetary radius

In [9]:
def Re_Computation_Planet_Radius(Nr_p,R_p,W1,M_p,rho):
    
    R_pr = np.zeros(len(Nr_p))
    
    #Loop of recomputation
    for i in range(0,len(Nr_p)):
        
        #Test si le rayon est connu
        if math.isnan(R_p[i]) is False:
            R_pr[i] = R_p[i]

        #Rayon inconnu : Recalcul du rayon 
        else:
            
            if W1[i].find('Kepler') == 0 and (M_p[i]/M_Terre)<= 16 :
                #Relation de Wolgang
                R_pr[i] = pow( M_p[i]/(2.7*M_Terre) , 10./13)*R_Terre 
                
                
            elif M_p[i]<7*M_Terre:
                
                if math.isnan(rho[i]) is True:
                    R_pr[i] = pow( (3./(4*np.pi) ) * (M_p[i]/rho_Terre),1./3)
                else:
                    R_pr[i] = pow( (3./(4*np.pi) ) * (M_p[i]/rho[i]),1./3)
            
            else :
                
                if math.isnan(rho[i]) is True:
                    R_pr[i] = pow( (3./(4*np.pi) ) * (M_p[i]/rho_J),1./3)
                else:
                    R_pr[i] = pow( (3./(4*np.pi) ) * (M_p[i]/rho[i]),1./3)
                    
    return R_pr

# Recomputation of the Planetary effective temperature 

In [10]:
def Re_Computation_Planet_Eff_Temp(Nr_p,T_p,R_s,S_p,T_s):
    
    T_pr  = np.zeros(len(Nr_p))
    
    for i in range(0,len(Nr_p)):
        
        if math.isnan(T_p[i]) is False:
            T_pr[i] = T_p[i]
        
        else:
            T_pr[i] = pow(  ( R_s[i]**2 * (1-A_B) ) / (4*S_p[i]**2)  ,   1./4  ) * T_s[i]
    
    return T_pr

# Thermal Flux Functions

In [11]:
def Flux_Stellar(L,T_s,R_s,D_s):
    c = 3E8
    nu = c/L 
    a = np.zeros(len(T_s))
    for i in range(0,len(a)):
        a[i] = 1E26*2*(h*nu**3/c**2)*pow(np.exp((h*nu)/(k_b*T_s[i])) -1, -1) * (R_s[i]/D_s[i])**2 *np.pi
    return a

In [12]:
def Flux_Planet(L,Nr_p,T_pr,R_pr,D_s):
    
    nu = c/L 
    
    a = np.zeros(len(Nr_p))

    for i in range(0,len(Nr_p)):
        a[i] = 1E26*2*(h*nu**3/c**2)*pow(np.exp((h*nu)/(k_b*T_pr[i])) -1, -1) *(R_pr[i]/D_s[i])**2 *np.pi
    
    return a

In [13]:
# FONCTION RENVOYANT LE NOMBRE DE LIGNES DE COMMENTAIRES
def countLigne(fichier):

    Liste=open(fichier,'r')
    i=1
    Ligne=Liste.readline()
    # "Tant que la ligne n'est pas égale à "" "
    #  ==> tant qu'on est pas arrivé à la fin 
    while Ligne[0] == "#":
        #on lit une ligne
        Ligne=Liste.readline()
        #on ajoute 1 à notre compteur
        i+=1

    #on retourne le compteur
    return i

# Integration time function (days)

In [14]:
def Integration_time(Nr_p,F_p):
    
    Time = np.array( np.zeros( len(Nr_p) ) )
    
    for i in range(0,len(Time)):
        Time[i] = S/F_p[i]
        Time[i] /= Trans
    
    return Time

# Main function displaying targets

In [15]:
def Display_Stars(D,df2,C,Time,fichier_1,hem):
    
    fichier_1.write("\n\n########################################################\n")

    fichier_1.write( "HOSTS STARS\n")

    fichier_1.write("\n"+ hem + "ERN HEMISPHERE\n")

    fichier_1.write( "Parsec" + '\t' + "\t" +'\t' + "Magnitude" + '\t\t' + 'RA' + '\t\t' + 'DEC' + '\t\t\t' + "Star")

    for i in range(0,len(D[0])):

        if math.isnan(D[9][i]) is True:
            #D[9][i] = 0
            df2.loc[i,'st_optmag']= 0

        if hem == 'NORTH':
            
            if math.isnan(C[i]) is False and C[i]>C_lim and (0 < D[6][i] <= 90) and D[17][i]>=R and  Time[i]<T_max:
                fichier_1.write(
                    '\n' 
                    + str("%.2e" % (D[10][i]/pc)) 
                    + "\t" 
                    + str(df2.loc[:,'st_optband'][i])  
                    + '\t' 
                    + str("%.2e" % df2.loc[:,'st_optmag'][i]) 
                    + '\t' 
                    + str("%.2e"%D[5][i]) 
                    + '\t' 
                    + str("%.2e"%D[6][i]) 
                    + '\t' 
                    + D[13][i])
            
                
        if hem == 'SOUTH':
            
            if math.isnan(C[i]) is False and C[i]>C_lim and (-90 < D[6][i] <= 0) and D[17][i]>=R and  Time[i]<T_max:
                fichier_1.write(
                    '\n' 
                    + str("%.2e" % (D[10][i]/pc)) 
                    + "\t" 
                    + str(df2.loc[:,'st_optband'][i])  
                    + '\t' 
                    + str("%.2e" % df2.loc[:,'st_optmag'][i]) 
                    + '\t' 
                    + str("%.2e"%D[5][i]) 
                    + '\t' 
                    + str("%.2e"%D[6][i]) 
                    + '\t' 
                    + D[13][i])
            
            

In [16]:
def Display_Planets(D,C,Time,fichier_1,F_p,F_rp,hem):
    
    if hem == 'NORTH':
        a = 0
        b = 90
    if hem == 'SOUTH':
        a = -90
        b = 0
    
    fichier_1.write("\n\n########################################################")
    
    fichier_1.write( "\n\nEXOPLANETS ORBITING HOSTS STARS\n")
    
    fichier_1.write("\n"+ hem + "ERN HEMISPHERE\n")
    
    fichier_1.write( "R_J" + '\t\t\t' + "°K" + '\t\t\t' + "mas" + '\t\t\t' + 'FLux(Jy)' + '\t' + 'R Flux(Jy)' + '\t' + 'Contrast' + '\t')
    
    for i in range(0,len(D[0])):

        if math.isnan(D[3][i]) is True:
            D[3][i] = 0
    
        if math.isnan(D[4][i]) is True:
            D[4][i] = 0

        
        if math.isnan(C[i]) is False and C[i]>C_lim and (a < D[6][i] <= b) and D[17][i]>=R and Time[i]<T_max:

            fichier_1.write('\n' 
                            
                + str("%.2e" % (D[3][i]/RJ)) 
                +'\t'
                + str("%.2e" % D[4][i]) 
                + '\t'  
                + str("%.2e" % D[17][i]) 
                +'\t'
                + str("%.2e" % F_p[i]) 
                + '\t'
                + str("%.2e" % F_rp[i]) 
                + '\t'
                + str("%.2e" % C[i])
                + '\t' 
                + str(D[7][i]) 
                + '\t' 
                +  D[13][i])